In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd 
%cd drive/MyDrive/models/SPQ/

/content
/content/drive/MyDrive/models/SPQ


In [4]:
!pip install kornia

     |████████████████████████████████| 474 kB 7.1 MB/s 


In [40]:
zz=[]
zz2=[]

In [6]:
from utils import *
from tqdm import tqdm
from dataset import SingleData
import math

def get_data_list2(data_path, ratio=0.95):
    print("*&$%#$%@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)
                # print("dir=>",dirs)

    np.random.seed(1)
    # train_img_list = np.random.choice(img_list, size=int(len(img_list)*(1-ratio)), replace=False)
    train_img_list=img_list
    #print(img_list, train_img_list)
    eval_img_list = list(set(img_list) - set(train_img_list))
    ########add
    # half=math.floor(len(eval_img_list)/2)
    # print(half)
    # eval_=eval_img_list[:half]
    # test_=eval_img_list[half:]
    #######
    #return class_name, train_img_list, eval_img_list 
    return class_name, train_img_list

def pqDist_one2(C, N_books, g_x, q_x,query_pname,gallery_pname):
    l1, l2 = C.shape
    L_word = int(l2/N_books)
    D_C = T.zeros((l1, N_books), dtype=T.float32)

    q_x_split = T.split(q_x, L_word, 0)
    g_x_split = np.split(g_x.cpu().data.numpy(), N_books, 1)
    C_split = T.split(C, L_word, 1)
    D_C_split = T.split(D_C, 1, 1)

    for j in range(N_books):
        for k in range(l1):
            # D_C_split[j][k] =T.norm(q_x_split[j]-C_split[j][k], 2)
            D_C_split[j][k] = T.norm(q_x_split[j]-C_split[j][k], 2).detach() #for PyTorch version over 1.9
        if j == 0:
            dist = D_C_split[j][g_x_split[j]]
        else:
            dist = T.add(dist, D_C_split[j][g_x_split[j]])
    Dpq = T.squeeze(dist)
    # ##### TASK
    # for i in range(len(gallery_pname)):
    #     if query_pname==gallery_pname[i]:
    #         Dpq[i]=float('inf')
    #         # print(query_pname,gallery_pname[i])
    # #####
    return Dpq

def Indexing(C, N_books, X):
    l1, l2 = C.shape
    L_word = int(l2/N_books)
    x = T.split(X, L_word, 1)
    y = T.split(C, L_word, 1)
    for i in range(N_books):
        diff = squared_distances(x[i], y[i])
        arg = T.argmin(diff, dim=1)
        min_idx = T.reshape(arg, [-1, 1])
        if i == 0:
            quant_idx = min_idx
        else:
            quant_idx = T.cat((quant_idx, min_idx), dim=1)
    return quant_idx

def Evaluate_mAP2(C, N_books, gallery_codes, query_codes, gallery_labels, query_labels, device,gallery_pname,query_pname, TOP_K=None):
    num_query = query_labels.shape[0]
    mean_AP = 0.0
    mean_acc = 0.0
    with tqdm(total=num_query, desc="Evaluate mAP", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
        print("@@@@ Gallery y",gallery_labels)
        print("@@@@ Gallery pname",gallery_pname)
        print("$$$$",gallery_labels[9876],gallery_pname[9876])
        for i in range(num_query):
            
            # Retrieve images from database
            retrieval = (query_labels[i, :] @ gallery_labels.t() > 0).float()

            # Arrange position according to hamming distance
            cc=T.argsort(pqDist_one2(C, N_books, gallery_codes, query_codes[i],query_pname[i],gallery_pname))

            retrieval = retrieval[cc][:TOP_K]
            # cc=gallery_pname[T.argsort(pqDist_one2(C, N_books, gallery_codes, query_codes[i],query_pname[i],gallery_pname))][:TOP_K]
            print("query_name",query_pname[i],query_labels[i, :])
            print("argsort")
            print(cc)
            # gallery_pname=gallery_pname[::-1]
            print("paitent num0",gallery_pname[cc[0]])
            print("paitent num1",gallery_pname[cc[1]])
            print("paitent num2",gallery_pname[cc[2]])
            print("paitent num3",gallery_pname[cc[3]])
            print("paitent num4",gallery_pname[cc[4]])
            print("paitent num4",gallery_pname[cc[5]])
            print("paitent num4",gallery_pname[cc[6]])
            print("paitent num4",gallery_pname[cc[7]])

            
            # print("paitent num1",gallery_labels[cc[-1]])
            # print("paitent num2",gallery_labels[cc[-2]])
            # print("paitent num3",gallery_labels[cc[-3]])
            # print("paitent num4",gallery_labels[cc[-4]])
            # print("paitent num5",gallery_labels[cc[-5]])
            # print("paitent num6",gallery_labels[cc[-6]])
            # print("paitent num7",gallery_labels[cc[-7]])
            # print("paitent num8",gallery_labels[cc[-8]])



            
            print("RETRival=>",retrieval)
            retrival_acc=retrieval[:1]
            # Retrieval count
            retrieval_cnt = retrieval.sum().int().item()
            retrieval_acc_cnt = retrival_acc.sum().int().item()
            print('retrieval_cnt=>',retrieval_acc_cnt)
            if retrieval_acc_cnt >0:
                mean_acc +=1
            # Can not retrieve images
            if retrieval_cnt == 0:
                continue

            # Generate score for every position
            score = T.linspace(1, retrieval_cnt, retrieval_cnt).to(device)

            # Acquire index
            index = (T.nonzero(retrieval == 1, as_tuple=False).squeeze() + 1.0).float().to(device)

            mean_AP += (score / index).mean()
            pbar.update(1)

        mean_AP = mean_AP / num_query
        mean_acc=mean_acc/num_query
    return mean_AP,mean_acc

def DoRetrieval2(device, args, net, C):
    print("Do Retrieval!!@!")


    class_name, train_img_list = get_data_list2('/content/drive/MyDrive/dataSets/CRC/train/',0.0)
    # class_name, train_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\train\\final\\final',0.0)
    train_data_set=SingleData(class_name, train_img_list, transform=transforms.ToTensor())
    Gallery_loader = T.utils.data.DataLoader(train_data_set, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

    # trainset = torchvision.datasets.CIFAR10(root=args.data_dir, train=True, download=args.if_download, transform=transforms.ToTensor())
    # Gallery_loader = T.utils.data.DataLoader(trainset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)
    class_test_name, test_img_list = get_data_list2('/content/drive/MyDrive/dataSets/CRC/heatmap test/',0.0)
    print("class test",class_test_name)
    print("img test",test_img_list)

    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final96\\diffrenet test\\final',0.9)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\same test\\fixed',0.0)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\same test\\fixed',0.0)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\diffrent test\\final\\fixed',0.0)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\same test\\fixed',0.8)
    test_data_set = SingleData(class_test_name, test_img_list, transform=transforms.ToTensor())    
    
    # testset = torchvision.datasets.CIFAR10(root=args.data_dir, train=False, download=args.if_download, transform=transforms.ToTensor())
    Query_loader = T.utils.data.DataLoader(test_data_set, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

    net.eval()
    with T.no_grad():
        with tqdm(total=len(Gallery_loader), desc="Build Gallery", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
            for i, data in enumerate(Gallery_loader, 0):
                gallery_x_batch, gallery_y_batch = data[0].to(device), data[1].to(device)
                gallery_y_batch=gallery_y_batch.long()
                outputs = net(gallery_x_batch)
                gallery_c_batch = Indexing(C, args.N_books, outputs[0])
                gallery_y_batch = T.eye(args.num_cls)[gallery_y_batch]
                temp=[]
                for s in data[2]:
                    index1=s.rfind('/')
                    # temp.append(s[index1+4:index1+7])
                    temp.append(s[index1:])
                    
                if i == 0:
                    gallery_c = gallery_c_batch
                    gallery_y = gallery_y_batch
                    gallery_pname=temp
                else:
                    gallery_c = T.cat([gallery_c, gallery_c_batch], 0)
                    gallery_y = T.cat([gallery_y, gallery_y_batch], 0)
                    gallery_pname=gallery_pname + temp
                
                # print(gallery_y_batch)
                # print(temp)
                pbar.update(1)
        # print('##############=>')
        # zz=gallery_pname
        # zz.append(3)
        # print("galley_label",gallery_y)

        # print("galley_pname",gallery_pname)
        with tqdm(total=len(Query_loader), desc="Compute Query", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
            for i, data in enumerate(Query_loader, 0):
              
                query_x_batch, query_y_batch = data[0].to(device), data[1].to(device)
                
                query_y_batch=query_y_batch.long()
                outputs = net(query_x_batch)
                query_y_batch = T.eye(args.num_cls)[query_y_batch]
                print("%%%%2",query_y_batch,data[2])
                temp=[]
                for s in data[2]:
                    index1=s.rfind('/')
                    # temp.append(s[index1+4:index1+7])
                    temp.append(s[index1:])
                    
                
                if i == 0:
                    query_c = outputs[0]
                    query_y = query_y_batch
                    query_pname=temp
                else:
                    query_c = T.cat([query_c, outputs[0]], 0)
                    query_y = T.cat([query_y, query_y_batch], 0)
                    query_pname=query_pname+temp
                pbar.update(1)
    print('##############=>')
    print("query_label",query_y)
    print("query_pname",query_pname)
    mAP,acc = Evaluate_mAP2(C, args.N_books, gallery_c.type(T.int), query_c, gallery_y, query_y, device,gallery_pname,query_pname, args.Top_N)
    print(mAP,acc)
    return mAP,acc


def get_args_parser():
    parser = argparse.ArgumentParser('SPQ', add_help=False)

    parser.add_argument('--gpu_id', default="0", type=str, help="""Define GPU id.""")
    parser.add_argument('--if_download', default=False, type=bool, help="""Whether to download the dataset or not.""")
    parser.add_argument('--data_dir', default="./data", type=str, help="""Path of the dataset to be installed.""")
    # parser.add_argument('--batch_size', default=256, type=int, help="""Training mini-batch size.""")
    parser.add_argument('--batch_size', default=32, type=int, help="""Training mini-batch size.""")
    parser.add_argument('--num_workers', default=8, type=int, help="""Number of data loading workers per GPU.""")
    parser.add_argument('--input_size', default=216, type=int, help="""Input image size, default is set to CIFAR10.""")

    parser.add_argument('--N_books', default=8, type=int, help="""The number of the codebooks.""")
    parser.add_argument('--N_words', default=16, type=int, help="""The number of the codewords. It should be a power of two.""")
    parser.add_argument('--L_word', default=16, type=int, help="""Dimensionality of the codeword.""")
    parser.add_argument('--soft_quantization_scale', default=5.0, type=float, help="""Soft-quantization scaling parameter.""")
    parser.add_argument('--contrastive_temperature', default=0.5, type=float, help="""Contrastive learning Temperature scaling parameter.""")
    
    parser.add_argument('--num_cls', default="3", type=int, help="""The number of classes in the dataset for evaluation, default is set to CIFAR10""")
    parser.add_argument('--eval_epoch', default=50, type=int, help="""Compute mAP for Every N-th epoch.""")
    # parser.add_argument('--eval_epoch', default=100, type=int, help="""Compute mAP for Every N-th epoch.""")
    parser.add_argument('--output_dir', default=".", type=str, help="""Path to save logs and checkpoints.""")
    parser.add_argument('--Top_N', default=8, type=int, help="""Top N number of images to be retrieved for evaluation.""")
    
    return parser
class CQCLoss(T.nn.Module):

    def __init__(self, device, batch_size, tau_cqc):
        super(CQCLoss, self).__init__()
        self.batch_size = batch_size
        self.tau_cqc = tau_cqc
        self.device = device
        self.COSSIM = T.nn.CosineSimilarity(dim=-1)
        self.CE = T.nn.CrossEntropyLoss(reduction="sum")
        self.get_corr_mask = self._get_correlated_mask().type(T.bool)

    def _get_correlated_mask(self):
        diag = np.eye(2 * self.batch_size)
        l1 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=-self.batch_size)
        l2 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=self.batch_size)
        mask = T.from_numpy((diag + l1 + l2))
        mask = (1 - mask).type(T.bool)
        return mask.to(self.device)

    def forward(self, Xa, Xb, Za, Zb):

        XaZb = T.cat([Xa, Zb], dim=0)
        XbZa = T.cat([Xb, Za], dim=0)

        Cossim_ab = self.COSSIM(XaZb.unsqueeze(1), XaZb.unsqueeze(0))
        Rab = T.diag(Cossim_ab, self.batch_size)
        Lab = T.diag(Cossim_ab, -self.batch_size)
        Pos_ab = T.cat([Rab, Lab]).view(2 * self.batch_size, 1)
        Neg_ab = Cossim_ab[self.get_corr_mask].view(2 * self.batch_size, -1)

        Cossim_ba = self.COSSIM(XbZa.unsqueeze(1), XbZa.unsqueeze(0))
        Rba = T.diag(Cossim_ba, self.batch_size)
        Lba = T.diag(Cossim_ba, -self.batch_size)    
        Pos_ba = T.cat([Rba, Lba]).view(2 * self.batch_size, 1)
        Neg_ba = Cossim_ba[self.get_corr_mask].view(2 * self.batch_size, -1)


        logits_ab = T.cat((Pos_ab, Neg_ab), dim=1)
        logits_ab /= self.tau_cqc

        logits_ba = T.cat((Pos_ba, Neg_ba), dim=1)
        logits_ba /= self.tau_cqc

        labels = T.zeros(2 * self.batch_size).to(self.device).long()
        
        loss = self.CE(logits_ab, labels) + self.CE(logits_ba, labels)
        return loss / (2 * self.batch_size)




from torch.optim.lr_scheduler import CosineAnnealingLR
from utils import *
from Retrieval import *
from dataset import SingleData
import math
parser = argparse.ArgumentParser('SPQ', parents=[get_args_parser()])
args = parser.parse_args("")


if __name__ == '__main__':
    
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_id
    device = T.device('cuda')
    
    sz = args.input_size
    data_dir = args.data_dir
    batch_size = args.batch_size
    
    N_books = args.N_books
    N_words = args.N_words
    L_word = args.L_word
    tau_q = args.soft_quantization_scale
    tau_cqc = args.contrastive_temperature
    
    N_bits = int(N_books * np.sqrt(N_words))
    print('\033[91m' + '%d'%N_bits +  '-bit to retrieval' + '\033[0m')
    
    #Define the data augmentation following the setup of SimCLR
    Augmentation = nn.Sequential(
        Kg.RandomResizedCrop(size=(sz, sz)),
        Kg.RandomHorizontalFlip(p=0.5),
        Kg.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1, p=0.8),
        Kg.RandomGrayscale(p=0.2),
        Kg.RandomGaussianBlur((int(0.1 * sz), int(0.1 * sz)), (0.1, 2.0), p=0.5))
    
    transform = transforms.ToTensor()
    
    class_name, train_img_list = get_data_list2('/content/drive/MyDrive/dataSets/CRC/train/')
    train_data_set=SingleData(class_name, train_img_list, transform)
    
    # trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=args.if_download, transform=transform)
    trainloader = T.utils.data.DataLoader(train_data_set, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=args.num_workers)
    # trainloader = T.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=args.num_workers)
    # testset = torchvision.datasets.CIFAR10(root=args.data_dir, train=False, download=args.if_download, transform=transforms.ToTensor())
    class Quantization_Head(nn.Module):
        def __init__(self, N_words, N_books, L_word, tau_q):
            super(Quantization_Head, self).__init__()
            self.fc = nn.Linear(512, N_books * L_word)
            nn.init.xavier_uniform_(self.fc.weight)
    
            # Codebooks
            self.C = T.nn.Parameter(Variable((T.randn(N_words, N_books * L_word)).type(T.float32), requires_grad=True))
            nn.init.xavier_uniform_(self.C)
    
            self.N_books = N_books
            self.L_word = L_word
            self.tau_q = tau_q
    
        def forward(self, input):
            X = self.fc(input)
            Z = Soft_Quantization(X, self.C, self.N_books, self.tau_q)
            return X, Z
    
    
    Q = Quantization_Head(N_words, N_books, L_word, tau_q)
    net = nn.Sequential(ResNet_Baseline(BasicBlock, [2, 2, 2, 2]), Q)
    net.load_state_dict(T.load('/content/drive/MyDrive/models/SPQ/199_32_0.1983_checkpoint.pth'))
    net.cuda(device)
    
    criterion = CQCLoss(device, batch_size, tau_cqc)
    
    optimizer = T.optim.Adam(net.parameters(), lr=3e-4, weight_decay=10e-6)
    scheduler = CosineAnnealingLR(optimizer, T_max=len(trainloader), eta_min=0, last_epoch=-1)
    
    MAX_mAP = 0.0
    mAP = 0.0
    
    
    mAP,acc = DoRetrieval2(device, args, net, Q.C)

32-bit to retrieval
*&$%#$%@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
['s1', 's2', 's3']


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Do Retrieval!!@!
*&$%#$%@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
['s1', 's2', 's3']
*&$%#$%@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
class test ['s1', 's2', 's3']
img test ['/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_009_12_614.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_029_24_104.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_002_2_81.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_024_21_46.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_002_2_1014.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_022_19_411.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_050_32_312.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_022_20_710.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_038_29_109.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_002_3_41.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_002_3_39.png', '/content/drive/MyDrive/dataSe

Compute Query   33%|███▎      | 1/3 [00:00<00:01,  1.16it/s]

%%%%2 tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]]) ('/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_009_12_614.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_029_24_104.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_002_2_81.png', '/content/drive/MyDrive/dataSets/CRC/heatmap test/s1/s1_024_21_46.png',

...

paitent num2 /s1_161_90_410.png
paitent num3 /s3_177_58_1016.png
paitent num4 /s1_167_95_911.png
paitent num4 /s1_156_66_10.png
paitent num4 /s2_041_50_910.png
paitent num4 /s3_162_26_147.png
RETRival=> tensor([1., 1., 0., 1., 0., 0., 0., 1.])
retrieval_cnt=> 1
query_name /s3_167_29_522.png tensor([0., 0., 1.])
argsort
tensor([11247,  9590, 14642,  ..., 10286,  1729,  5708])
paitent num0 /s3_172_41_72.png
paitent num1 /s2_008_10_87.png
paitent num2 /s3_083_15_140.png
paitent num3 /s3_176_52_625.png
paitent num4 /s3_175_46_612.png
paitent num4 /s1_007_8_65.png
paitent num4 /s3_175_46_614.png
paitent num4 /s3_167_28_827.png
RETRival=> tensor([1., 0., 1., 1., 1., 0., 1., 1.])
retrieval_cnt=> 1
query_name /s3_137_23_1415.png tensor([0., 0., 1.])
argsort
tensor([13496,  4386,  4379,  ...,   981, 13105,  9863])
paitent num0 /s3_137_23_1312.png
paitent num1 /s1_007_8_1110.png
paitent num2 /s1_007_8_1012.png
paitent num3 /s3_162_27_163.png
paitent num4 /s3_162_27_317.png
paitent num4 /s3_137_2

Evaluate mAP    90%|█████████ | 65/72 [00:01<00:00, 41.66it/s]

tensor([0., 0., 1.])
argsort
tensor([14692, 14445, 13690,  ...,  5060,  7902,  1411])
paitent num0 /s3_083_15_2319.png
paitent num1 /s3_079_13_1919.png
paitent num2 /s3_149_25_179.png
paitent num3 /s1_135_46_43.png
paitent num4 /s1_052_34_914.png
paitent num4 /s1_151_52_88.png
paitent num4 /s1_012_17_015.png
paitent num4 /s1_157_73_913.png
RETRival=> tensor([1., 1., 1., 0., 0., 0., 0., 0.])
retrieval_cnt=> 1
query_name /s3_084_18_92.png tensor([0., 0., 1.])
argsort
tensor([ 3163,  1312,  3648,  ...,  2577,   926, 11345])
paitent num0 /s1_026_23_715.png
paitent num1 /s1_155_65_415.png
paitent num2 /s1_050_32_99.png
paitent num3 /s2_054_77_1114.png
paitent num4 /s2_054_76_06.png
paitent num4 /s1_056_38_97.png
paitent num4 /s1_157_72_916.png
paitent num4 /s1_035_27_34.png
RETRival=> tensor([0., 0., 0., 0., 0., 0., 0., 0.])
retrieval_cnt=> 0
query_name /s3_085_19_110.png tensor([0., 0., 1.])
argsort
tensor([ 2008,  5791, 12125,  ...,  7157,  5093,  1219])
paitent num0 /s1_059_39_212.png
pa

Evaluate mAP    96%|█████████▌| 69/72 [00:01<00:00, 39.34it/s]

query_name /s3_085_19_155.png tensor([0., 0., 1.])
argsort
tensor([13639, 14643, 13641,  ...,  6268, 12354,  5249])
paitent num0 /s3_149_25_627.png
paitent num1 /s3_083_15_1414.png
paitent num2 /s3_149_25_618.png
paitent num3 /s1_154_60_32.png
paitent num4 /s3_149_25_1212.png
paitent num4 /s2_028_28_06.png
paitent num4 /s3_085_19_154.png
paitent num4 /s3_175_47_019.png
RETRival=> tensor([1., 1., 1., 0., 1., 0., 1., 1.])
retrieval_cnt=> 1
query_name /s3_083_15_22.png tensor([0., 0., 1.])
argsort
tensor([13592, 12746,  7384,  ...,  9270,  9772,  3052])
paitent num0 /s3_137_24_1331.png
paitent num1 /s3_172_36_731.png
paitent num2 /s2_040_47_21.png
paitent num3 /s3_172_37_1630.png
paitent num4 /s1_052_34_101.png
paitent num4 /s3_176_52_17.png
paitent num4 /s1_155_62_31.png
paitent num4 /s3_175_48_826.png
RETRival=> tensor([1., 1., 0., 1., 0., 1., 0., 1.])
retrieval_cnt=> 1
query_name /s3_083_14_1325.png tensor([0., 0., 1.])
argsort
tensor([ 8546, 10874, 10833,  ...,  9833,  6786,  5625])
p

In [50]:
zz

[]

for results

In [20]:
from utils import *
from tqdm import tqdm
from dataset import SingleData
import math

def get_data_list(data_path, ratio=0.95):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list = np.random.choice(img_list, size=int(len(img_list)*(1-ratio)), replace=False)
    #print(img_list, train_img_list)
    eval_img_list = list(set(img_list) - set(train_img_list))
    ########add
    half=math.floor(len(eval_img_list)/2)
    print(half)
    eval_=eval_img_list[:half]
    test_=eval_img_list[half:]
    #######
    #return class_name, train_img_list, eval_img_list 
    return class_name, train_img_list

def pqDist_one2(C, N_books, g_x, q_x,query_pname,gallery_pname):
    l1, l2 = C.shape
    L_word = int(l2/N_books)
    D_C = T.zeros((l1, N_books), dtype=T.float32)

    q_x_split = T.split(q_x, L_word, 0)
    g_x_split = np.split(g_x.cpu().data.numpy(), N_books, 1)
    C_split = T.split(C, L_word, 1)
    D_C_split = T.split(D_C, 1, 1)

    for j in range(N_books):
        for k in range(l1):
            # D_C_split[j][k] =T.norm(q_x_split[j]-C_split[j][k], 2)
            D_C_split[j][k] = T.norm(q_x_split[j]-C_split[j][k], 2).detach() #for PyTorch version over 1.9
        if j == 0:
            dist = D_C_split[j][g_x_split[j]]
        else:
            dist = T.add(dist, D_C_split[j][g_x_split[j]])
    Dpq = T.squeeze(dist)
    # ##### TASK
    # for i in range(len(gallery_pname)):
    #     if query_pname==gallery_pname[i]:
    #         Dpq[i]=float('inf')
    #         # print(query_pname,gallery_pname[i])
    # #####
    return Dpq

def Indexing(C, N_books, X):
    l1, l2 = C.shape
    L_word = int(l2/N_books)
    x = T.split(X, L_word, 1)
    y = T.split(C, L_word, 1)
    for i in range(N_books):
        diff = squared_distances(x[i], y[i])
        arg = T.argmin(diff, dim=1)
        min_idx = T.reshape(arg, [-1, 1])
        if i == 0:
            quant_idx = min_idx
        else:
            quant_idx = T.cat((quant_idx, min_idx), dim=1)
    return quant_idx

def Evaluate_mAP2(C, N_books, gallery_codes, query_codes, gallery_labels, query_labels, device,gallery_pname,query_pname, TOP_K=None):
    num_query = query_labels.shape[0]
    mean_AP = 0.0
    mean_acc = 0.0
    with tqdm(total=num_query, desc="Evaluate mAP", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
        for i in range(num_query):
            
            # Retrieve images from database
            retrieval = (query_labels[i, :] @ gallery_labels.t() > 0).float()

            # Arrange position according to hamming distance
            retrieval = retrieval[T.argsort(pqDist_one2(C, N_books, gallery_codes, query_codes[i],query_pname[i],gallery_pname))][:TOP_K]
            # cc=gallery_pname[T.argsort(pqDist_one2(C, N_books, gallery_codes, query_codes[i],query_pname[i],gallery_pname))][:TOP_K]
            print("argsort")
            cc=T.argsort(pqDist_one2(C, N_books, gallery_codes, query_codes[i],query_pname[i],gallery_pname))
            print(cc)
            print("paitent num0",gallery_pname[cc[0]])
            print("paitent num1",gallery_pname[cc[1]])
            print("paitent num2",gallery_pname[cc[2]])
            print("paitent num3",gallery_pname[cc[3]])
            print("paitent num4",gallery_pname[cc[4]])


            
            print("RETRival=>",retrieval)
            retrival_acc=retrieval[:1]
            # Retrieval count
            retrieval_cnt = retrieval.sum().int().item()
            retrieval_acc_cnt = retrival_acc.sum().int().item()
            print('retrieval_cnt=>',retrieval_acc_cnt)
            if retrieval_acc_cnt >0:
                mean_acc +=1
            # Can not retrieve images
            if retrieval_cnt == 0:
                continue

            # Generate score for every position
            score = T.linspace(1, retrieval_cnt, retrieval_cnt).to(device)

            # Acquire index
            index = (T.nonzero(retrieval == 1, as_tuple=False).squeeze() + 1.0).float().to(device)

            mean_AP += (score / index).mean()
            pbar.update(1)

        mean_AP = mean_AP / num_query
        mean_acc=mean_acc/num_query
    return mean_AP,mean_acc

def DoRetrieval2(device, args, net, C):
    print("Do Retrieval!!@!")


    class_name, train_img_list = get_data_list('/content/drive/MyDrive/dataSets/CRC/train/',0.0)
    # class_name, train_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\train\\final\\final',0.0)
    train_data_set=SingleData(class_name, train_img_list, transform=transforms.ToTensor())
    Gallery_loader = T.utils.data.DataLoader(train_data_set, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

    # trainset = torchvision.datasets.CIFAR10(root=args.data_dir, train=True, download=args.if_download, transform=transforms.ToTensor())
    # Gallery_loader = T.utils.data.DataLoader(trainset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)
    class_test_name, test_img_list = get_data_list('/content/drive/MyDrive/dataSets/CRC/temp heatmap tesnor/',0.0)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final96\\diffrenet test\\final',0.9)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\same test\\fixed',0.0)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\same test\\fixed',0.0)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\diffrent test\\final\\fixed',0.0)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\same test\\fixed',0.8)
    test_data_set = SingleData(class_test_name, test_img_list, transform=transforms.ToTensor())    
    
    # testset = torchvision.datasets.CIFAR10(root=args.data_dir, train=False, download=args.if_download, transform=transforms.ToTensor())
    Query_loader = T.utils.data.DataLoader(test_data_set, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

    net.eval()
    with T.no_grad():
        # with tqdm(total=len(Gallery_loader), desc="Build Gallery", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
        #     for i, data in enumerate(Gallery_loader, 0):
        #         gallery_x_batch, gallery_y_batch = data[0].to(device), data[1].to(device)
        #         gallery_y_batch=gallery_y_batch.long()
        #         outputs = net(gallery_x_batch)
        #         gallery_c_batch = Indexing(C, args.N_books, outputs[0])
        #         gallery_y_batch = T.eye(args.num_cls)[gallery_y_batch]
        #         temp=[]
        #         for s in data[2]:
        #             index1=s.rfind('/')
        #             # temp.append(s[index1+4:index1+7])
        #             temp.append(s[index1:])
                    
        #         if i == 0:
        #             gallery_c = gallery_c_batch
        #             gallery_y = gallery_y_batch
        #             gallery_pname=temp
        #         else:
        #             gallery_c = T.cat([gallery_c, gallery_c_batch], 0)
        #             gallery_y = T.cat([gallery_y, gallery_y_batch], 0)
        #             gallery_pname=gallery_pname + temp
        #         pbar.update(1)

        with tqdm(total=len(Query_loader), desc="Compute Query", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
            for i, data in enumerate(Query_loader, 0):
                query_x_batch, query_y_batch = data[0].to(device), data[1].to(device)
                query_y_batch=query_y_batch.long()
                outputs = net(query_x_batch)
                query_y_batch = T.eye(args.num_cls)[query_y_batch]
                
                temp=[]
                for s in data[2]:
                    index1=s.rfind('/')
                    # temp.append(s[index1+4:index1+7])
                    temp.append(s[index1:])
                    

                if i == 0:
                    query_c = outputs[0]
                    query_y = query_y_batch
                    query_pname=temp
                else:
                    query_c = T.cat([query_c, outputs[0]], 0)
                    query_y = T.cat([query_y, query_y_batch], 0)
                    query_pname=query_pname+temp
                pbar.update(1)
    return 1
    # mAP,acc = Evaluate_mAP2(C, args.N_books, gallery_c.type(T.int), query_c, gallery_y, query_y, device,gallery_pname,query_pname, args.Top_N)
    print(mAP,acc)
    return mAP,acc


def get_args_parser():
    parser = argparse.ArgumentParser('SPQ', add_help=False)

    parser.add_argument('--gpu_id', default="0", type=str, help="""Define GPU id.""")
    parser.add_argument('--if_download', default=False, type=bool, help="""Whether to download the dataset or not.""")
    parser.add_argument('--data_dir', default="./data", type=str, help="""Path of the dataset to be installed.""")
    # parser.add_argument('--batch_size', default=256, type=int, help="""Training mini-batch size.""")
    parser.add_argument('--batch_size', default=32, type=int, help="""Training mini-batch size.""")
    parser.add_argument('--num_workers', default=8, type=int, help="""Number of data loading workers per GPU.""")
    parser.add_argument('--input_size', default=216, type=int, help="""Input image size, default is set to CIFAR10.""")

    parser.add_argument('--N_books', default=8, type=int, help="""The number of the codebooks.""")
    parser.add_argument('--N_words', default=16, type=int, help="""The number of the codewords. It should be a power of two.""")
    parser.add_argument('--L_word', default=16, type=int, help="""Dimensionality of the codeword.""")
    parser.add_argument('--soft_quantization_scale', default=5.0, type=float, help="""Soft-quantization scaling parameter.""")
    parser.add_argument('--contrastive_temperature', default=0.5, type=float, help="""Contrastive learning Temperature scaling parameter.""")
    
    parser.add_argument('--num_cls', default="3", type=int, help="""The number of classes in the dataset for evaluation, default is set to CIFAR10""")
    parser.add_argument('--eval_epoch', default=50, type=int, help="""Compute mAP for Every N-th epoch.""")
    # parser.add_argument('--eval_epoch', default=100, type=int, help="""Compute mAP for Every N-th epoch.""")
    parser.add_argument('--output_dir', default=".", type=str, help="""Path to save logs and checkpoints.""")
    parser.add_argument('--Top_N', default=8, type=int, help="""Top N number of images to be retrieved for evaluation.""")
    
    return parser
class CQCLoss(T.nn.Module):

    def __init__(self, device, batch_size, tau_cqc):
        super(CQCLoss, self).__init__()
        self.batch_size = batch_size
        self.tau_cqc = tau_cqc
        self.device = device
        self.COSSIM = T.nn.CosineSimilarity(dim=-1)
        self.CE = T.nn.CrossEntropyLoss(reduction="sum")
        self.get_corr_mask = self._get_correlated_mask().type(T.bool)

    def _get_correlated_mask(self):
        diag = np.eye(2 * self.batch_size)
        l1 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=-self.batch_size)
        l2 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=self.batch_size)
        mask = T.from_numpy((diag + l1 + l2))
        mask = (1 - mask).type(T.bool)
        return mask.to(self.device)

    def forward(self, Xa, Xb, Za, Zb):

        XaZb = T.cat([Xa, Zb], dim=0)
        XbZa = T.cat([Xb, Za], dim=0)

        Cossim_ab = self.COSSIM(XaZb.unsqueeze(1), XaZb.unsqueeze(0))
        Rab = T.diag(Cossim_ab, self.batch_size)
        Lab = T.diag(Cossim_ab, -self.batch_size)
        Pos_ab = T.cat([Rab, Lab]).view(2 * self.batch_size, 1)
        Neg_ab = Cossim_ab[self.get_corr_mask].view(2 * self.batch_size, -1)

        Cossim_ba = self.COSSIM(XbZa.unsqueeze(1), XbZa.unsqueeze(0))
        Rba = T.diag(Cossim_ba, self.batch_size)
        Lba = T.diag(Cossim_ba, -self.batch_size)    
        Pos_ba = T.cat([Rba, Lba]).view(2 * self.batch_size, 1)
        Neg_ba = Cossim_ba[self.get_corr_mask].view(2 * self.batch_size, -1)


        logits_ab = T.cat((Pos_ab, Neg_ab), dim=1)
        logits_ab /= self.tau_cqc

        logits_ba = T.cat((Pos_ba, Neg_ba), dim=1)
        logits_ba /= self.tau_cqc

        labels = T.zeros(2 * self.batch_size).to(self.device).long()
        
        loss = self.CE(logits_ab, labels) + self.CE(logits_ba, labels)
        return loss / (2 * self.batch_size)




from torch.optim.lr_scheduler import CosineAnnealingLR
from utils import *
from Retrieval import *
from dataset import SingleData
import math
parser = argparse.ArgumentParser('SPQ', parents=[get_args_parser()])
args = parser.parse_args("")


if __name__ == '__main__':
    
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_id
    device = T.device('cuda')
    
    sz = args.input_size
    data_dir = args.data_dir
    batch_size = args.batch_size
    
    N_books = args.N_books
    N_words = args.N_words
    L_word = args.L_word
    tau_q = args.soft_quantization_scale
    tau_cqc = args.contrastive_temperature
    
    N_bits = int(N_books * np.sqrt(N_words))
    print('\033[91m' + '%d'%N_bits +  '-bit to retrieval' + '\033[0m')
    
    #Define the data augmentation following the setup of SimCLR
    Augmentation = nn.Sequential(
        Kg.RandomResizedCrop(size=(sz, sz)),
        Kg.RandomHorizontalFlip(p=0.5),
        Kg.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1, p=0.8),
        Kg.RandomGrayscale(p=0.2),
        Kg.RandomGaussianBlur((int(0.1 * sz), int(0.1 * sz)), (0.1, 2.0), p=0.5))
    
    transform = transforms.ToTensor()
    
    class_name, train_img_list = get_data_list('/content/drive/MyDrive/dataSets/CRC/train/')
    train_data_set=SingleData(class_name, train_img_list, transform)
    
    # trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=args.if_download, transform=transform)
    trainloader = T.utils.data.DataLoader(train_data_set, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=args.num_workers)
    # trainloader = T.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=args.num_workers)
    # testset = torchvision.datasets.CIFAR10(root=args.data_dir, train=False, download=args.if_download, transform=transforms.ToTensor())
    class Quantization_Head(nn.Module):
        def __init__(self, N_words, N_books, L_word, tau_q):
            super(Quantization_Head, self).__init__()
            self.fc = nn.Linear(512, N_books * L_word)
            nn.init.xavier_uniform_(self.fc.weight)
    
            # Codebooks
            self.C = T.nn.Parameter(Variable((T.randn(N_words, N_books * L_word)).type(T.float32), requires_grad=True))
            nn.init.xavier_uniform_(self.C)
    
            self.N_books = N_books
            self.L_word = L_word
            self.tau_q = tau_q
    
        def forward(self, input):
            X = self.fc(input)
            Z = Soft_Quantization(X, self.C, self.N_books, self.tau_q)
            return X, Z
    
    
    Q = Quantization_Head(N_words, N_books, L_word, tau_q)
    net = nn.Sequential(ResNet_Baseline(BasicBlock, [2, 2, 2, 2]), Q)
    net.load_state_dict(T.load('/content/drive/MyDrive/models/SPQ/199_32_0.1983_checkpoint.pth'))
    net.cuda(device)
    
    criterion = CQCLoss(device, batch_size, tau_cqc)
    
    optimizer = T.optim.Adam(net.parameters(), lr=3e-4, weight_decay=10e-6)
    scheduler = CosineAnnealingLR(optimizer, T_max=len(trainloader), eta_min=0, last_epoch=-1)
    
    MAX_mAP = 0.0
    mAP = 0.0
    
    
    mAP,acc = DoRetrieval2(device, args, net, Q.C)

32-bit to retrieval
7127
['s1', 's2', 's3']


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Do Retrieval!!@!
0
['s1', 's2', 's3']
0
['s1']


Compute Query  100%|██████████| 1/1 [00:00<00:00,  2.51it/s]


TypeError: ignored

In [7]:
from utils import *
from tqdm import tqdm
from dataset import SingleData
import math

def get_data_list2(data_path, ratio=0.95):
    print("*&$%#$%@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)
                # print("dir=>",dirs)

    np.random.seed(1)
    # train_img_list = np.random.choice(img_list, size=int(len(img_list)*(1-ratio)), replace=False)
    train_img_list=img_list
    #print(img_list, train_img_list)
    eval_img_list = list(set(img_list) - set(train_img_list))
    ########add
    # half=math.floor(len(eval_img_list)/2)
    # print(half)
    # eval_=eval_img_list[:half]
    # test_=eval_img_list[half:]
    #######
    #return class_name, train_img_list, eval_img_list 
    return class_name, train_img_list

def pqDist_one2(C, N_books, g_x, q_x,query_pname,gallery_pname):
    l1, l2 = C.shape
    L_word = int(l2/N_books)
    D_C = T.zeros((l1, N_books), dtype=T.float32)

    q_x_split = T.split(q_x, L_word, 0)
    g_x_split = np.split(g_x.cpu().data.numpy(), N_books, 1)
    C_split = T.split(C, L_word, 1)
    D_C_split = T.split(D_C, 1, 1)

    for j in range(N_books):
        for k in range(l1):
            # D_C_split[j][k] =T.norm(q_x_split[j]-C_split[j][k], 2)
            D_C_split[j][k] = T.norm(q_x_split[j]-C_split[j][k], 2).detach() #for PyTorch version over 1.9
        if j == 0:
            dist = D_C_split[j][g_x_split[j]]
        else:
            dist = T.add(dist, D_C_split[j][g_x_split[j]])
    Dpq = T.squeeze(dist)
    # ##### TASK
    # for i in range(len(gallery_pname)):
    #     if query_pname==gallery_pname[i]:
    #         Dpq[i]=float('inf')
    #         # print(query_pname,gallery_pname[i])
    # #####
    return Dpq

def Indexing(C, N_books, X):
    l1, l2 = C.shape
    L_word = int(l2/N_books)
    x = T.split(X, L_word, 1)
    y = T.split(C, L_word, 1)
    for i in range(N_books):
        diff = squared_distances(x[i], y[i])
        arg = T.argmin(diff, dim=1)
        min_idx = T.reshape(arg, [-1, 1])
        if i == 0:
            quant_idx = min_idx
        else:
            quant_idx = T.cat((quant_idx, min_idx), dim=1)
    return quant_idx

def Evaluate_mAP2(C, N_books, gallery_codes, query_codes, gallery_labels, query_labels, device,gallery_pname,query_pname, TOP_K=None):
    num_query = query_labels.shape[0]
    mean_AP = 0.0
    mean_acc = 0.0
    with tqdm(total=num_query, desc="Evaluate mAP", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
        print("@@@@ Gallery y",gallery_labels)
        print("@@@@ Gallery pname",gallery_pname)
        print("$$$$",gallery_labels[9876],gallery_pname[9876])
        for i in range(num_query):
            
            # Retrieve images from database
            retrieval = (query_labels[i, :] @ gallery_labels.t() > 0).float()

            # Arrange position according to hamming distance
            cc=T.argsort(pqDist_one2(C, N_books, gallery_codes, query_codes[i],query_pname[i],gallery_pname))

            retrieval = retrieval[cc][:TOP_K]
            # cc=gallery_pname[T.argsort(pqDist_one2(C, N_books, gallery_codes, query_codes[i],query_pname[i],gallery_pname))][:TOP_K]
            print("query_name",query_pname[i],query_labels[i, :])
            print("argsort")
            print(cc)
            # gallery_pname=gallery_pname[::-1]
            print("paitent num0",gallery_pname[cc[0]])
            print("paitent num1",gallery_pname[cc[1]])
            print("paitent num2",gallery_pname[cc[2]])
            print("paitent num3",gallery_pname[cc[3]])
            print("paitent num4",gallery_pname[cc[4]])
            print("paitent num4",gallery_pname[cc[5]])
            print("paitent num4",gallery_pname[cc[6]])
            print("paitent num4",gallery_pname[cc[7]])

            
            # print("paitent num1",gallery_labels[cc[-1]])
            # print("paitent num2",gallery_labels[cc[-2]])
            # print("paitent num3",gallery_labels[cc[-3]])
            # print("paitent num4",gallery_labels[cc[-4]])
            # print("paitent num5",gallery_labels[cc[-5]])
            # print("paitent num6",gallery_labels[cc[-6]])
            # print("paitent num7",gallery_labels[cc[-7]])
            # print("paitent num8",gallery_labels[cc[-8]])



            
            print("RETRival=>",retrieval)
            retrival_acc=retrieval[:1]
            # Retrieval count
            retrieval_cnt = retrieval.sum().int().item()
            retrieval_acc_cnt = retrival_acc.sum().int().item()
            print('retrieval_cnt=>',retrieval_acc_cnt)
            if retrieval_acc_cnt >0:
                mean_acc +=1
            # Can not retrieve images
            if retrieval_cnt == 0:
                continue

            # Generate score for every position
            score = T.linspace(1, retrieval_cnt, retrieval_cnt).to(device)

            # Acquire index
            index = (T.nonzero(retrieval == 1, as_tuple=False).squeeze() + 1.0).float().to(device)

            mean_AP += (score / index).mean()
            pbar.update(1)

        mean_AP = mean_AP / num_query
        mean_acc=mean_acc/num_query
    return mean_AP,mean_acc

def DoRetrieval2(device, args, net, C):
    print("Do Retrieval!!@!")


    class_name, train_img_list = get_data_list2('/content/drive/MyDrive/dataSets/CRC/train/',0.0)
    # class_name, train_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\train\\final\\final',0.0)
    train_data_set=SingleData(class_name, train_img_list, transform=transforms.ToTensor())
    Gallery_loader = T.utils.data.DataLoader(train_data_set, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

    # trainset = torchvision.datasets.CIFAR10(root=args.data_dir, train=True, download=args.if_download, transform=transforms.ToTensor())
    # Gallery_loader = T.utils.data.DataLoader(trainset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)
    class_test_name, test_img_list = get_data_list2('/content/drive/MyDrive/dataSets/CRC/temp heatmap tesnor/',0.0)
    print("class test",class_test_name)
    print("img test",test_img_list)

    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final96\\diffrenet test\\final',0.9)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\same test\\fixed',0.0)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\same test\\fixed',0.0)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\diffrent test\\final\\fixed',0.0)
    # class_test_name, test_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\same test\\fixed',0.8)
    test_data_set = SingleData(class_test_name, test_img_list, transform=transforms.ToTensor())    
    
    # testset = torchvision.datasets.CIFAR10(root=args.data_dir, train=False, download=args.if_download, transform=transforms.ToTensor())
    Query_loader = T.utils.data.DataLoader(test_data_set, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

    net.eval()
    with T.no_grad():
        # with tqdm(total=len(Gallery_loader), desc="Build Gallery", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
        #     for i, data in enumerate(Gallery_loader, 0):
        #         gallery_x_batch, gallery_y_batch = data[0].to(device), data[1].to(device)
        #         gallery_y_batch=gallery_y_batch.long()
        #         outputs = net(gallery_x_batch)
        #         gallery_c_batch = Indexing(C, args.N_books, outputs[0])
        #         gallery_y_batch = T.eye(args.num_cls)[gallery_y_batch]
        #         temp=[]
        #         for s in data[2]:
        #             index1=s.rfind('/')
        #             # temp.append(s[index1+4:index1+7])
        #             temp.append(s[index1:])
                    
        #         if i == 0:
        #             gallery_c = gallery_c_batch
        #             gallery_y = gallery_y_batch
        #             gallery_pname=temp
        #         else:
        #             gallery_c = T.cat([gallery_c, gallery_c_batch], 0)
        #             gallery_y = T.cat([gallery_y, gallery_y_batch], 0)
        #             gallery_pname=gallery_pname + temp
                
        #         # print(gallery_y_batch)
        #         # print(temp)
        #         pbar.update(1)
        # # print('##############=>')
        # # zz=gallery_pname
        # # zz.append(3)
        # # print("galley_label",gallery_y)

        # # print("galley_pname",gallery_pname)
        with tqdm(total=len(Query_loader), desc="Compute Query", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
            for i, data in enumerate(Query_loader, 0):
              
                query_x_batch, query_y_batch = data[0].to(device), data[1].to(device)
                
                query_y_batch=query_y_batch.long()
                outputs = net(query_x_batch)
                query_y_batch = T.eye(args.num_cls)[query_y_batch]
                print("%%%%2",query_y_batch,data[2])
                temp=[]
                for s in data[2]:
                    index1=s.rfind('/')
                    # temp.append(s[index1+4:index1+7])
                    temp.append(s[index1:])
                    
                
                if i == 0:
                    query_c = outputs[0]
                    query_y = query_y_batch
                    query_pname=temp
                else:
                    query_c = T.cat([query_c, outputs[0]], 0)
                    query_y = T.cat([query_y, query_y_batch], 0)
                    query_pname=query_pname+temp
                pbar.update(1)
    # print('##############=>')
    # print("query_label",query_y)
    # print("query_pname",query_pname)
    return 1
    # mAP,acc = Evaluate_mAP2(C, args.N_books, gallery_c.type(T.int), query_c, gallery_y, query_y, device,gallery_pname,query_pname, args.Top_N)
    print(mAP,acc)
    return mAP,acc


def get_args_parser():
    parser = argparse.ArgumentParser('SPQ', add_help=False)

    parser.add_argument('--gpu_id', default="0", type=str, help="""Define GPU id.""")
    parser.add_argument('--if_download', default=False, type=bool, help="""Whether to download the dataset or not.""")
    parser.add_argument('--data_dir', default="./data", type=str, help="""Path of the dataset to be installed.""")
    # parser.add_argument('--batch_size', default=256, type=int, help="""Training mini-batch size.""")
    parser.add_argument('--batch_size', default=32, type=int, help="""Training mini-batch size.""")
    parser.add_argument('--num_workers', default=8, type=int, help="""Number of data loading workers per GPU.""")
    parser.add_argument('--input_size', default=216, type=int, help="""Input image size, default is set to CIFAR10.""")

    parser.add_argument('--N_books', default=8, type=int, help="""The number of the codebooks.""")
    parser.add_argument('--N_words', default=16, type=int, help="""The number of the codewords. It should be a power of two.""")
    parser.add_argument('--L_word', default=16, type=int, help="""Dimensionality of the codeword.""")
    parser.add_argument('--soft_quantization_scale', default=5.0, type=float, help="""Soft-quantization scaling parameter.""")
    parser.add_argument('--contrastive_temperature', default=0.5, type=float, help="""Contrastive learning Temperature scaling parameter.""")
    
    parser.add_argument('--num_cls', default="3", type=int, help="""The number of classes in the dataset for evaluation, default is set to CIFAR10""")
    parser.add_argument('--eval_epoch', default=50, type=int, help="""Compute mAP for Every N-th epoch.""")
    # parser.add_argument('--eval_epoch', default=100, type=int, help="""Compute mAP for Every N-th epoch.""")
    parser.add_argument('--output_dir', default=".", type=str, help="""Path to save logs and checkpoints.""")
    parser.add_argument('--Top_N', default=8, type=int, help="""Top N number of images to be retrieved for evaluation.""")
    
    return parser
class CQCLoss(T.nn.Module):

    def __init__(self, device, batch_size, tau_cqc):
        super(CQCLoss, self).__init__()
        self.batch_size = batch_size
        self.tau_cqc = tau_cqc
        self.device = device
        self.COSSIM = T.nn.CosineSimilarity(dim=-1)
        self.CE = T.nn.CrossEntropyLoss(reduction="sum")
        self.get_corr_mask = self._get_correlated_mask().type(T.bool)

    def _get_correlated_mask(self):
        diag = np.eye(2 * self.batch_size)
        l1 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=-self.batch_size)
        l2 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=self.batch_size)
        mask = T.from_numpy((diag + l1 + l2))
        mask = (1 - mask).type(T.bool)
        return mask.to(self.device)

    def forward(self, Xa, Xb, Za, Zb):

        XaZb = T.cat([Xa, Zb], dim=0)
        XbZa = T.cat([Xb, Za], dim=0)

        Cossim_ab = self.COSSIM(XaZb.unsqueeze(1), XaZb.unsqueeze(0))
        Rab = T.diag(Cossim_ab, self.batch_size)
        Lab = T.diag(Cossim_ab, -self.batch_size)
        Pos_ab = T.cat([Rab, Lab]).view(2 * self.batch_size, 1)
        Neg_ab = Cossim_ab[self.get_corr_mask].view(2 * self.batch_size, -1)

        Cossim_ba = self.COSSIM(XbZa.unsqueeze(1), XbZa.unsqueeze(0))
        Rba = T.diag(Cossim_ba, self.batch_size)
        Lba = T.diag(Cossim_ba, -self.batch_size)    
        Pos_ba = T.cat([Rba, Lba]).view(2 * self.batch_size, 1)
        Neg_ba = Cossim_ba[self.get_corr_mask].view(2 * self.batch_size, -1)


        logits_ab = T.cat((Pos_ab, Neg_ab), dim=1)
        logits_ab /= self.tau_cqc

        logits_ba = T.cat((Pos_ba, Neg_ba), dim=1)
        logits_ba /= self.tau_cqc

        labels = T.zeros(2 * self.batch_size).to(self.device).long()
        
        loss = self.CE(logits_ab, labels) + self.CE(logits_ba, labels)
        return loss / (2 * self.batch_size)




from torch.optim.lr_scheduler import CosineAnnealingLR
from utils import *
from Retrieval import *
from dataset import SingleData
import math
parser = argparse.ArgumentParser('SPQ', parents=[get_args_parser()])
args = parser.parse_args("")


if __name__ == '__main__':
    
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_id
    device = T.device('cuda')
    
    sz = args.input_size
    data_dir = args.data_dir
    batch_size = args.batch_size
    
    N_books = args.N_books
    N_words = args.N_words
    L_word = args.L_word
    tau_q = args.soft_quantization_scale
    tau_cqc = args.contrastive_temperature
    
    N_bits = int(N_books * np.sqrt(N_words))
    print('\033[91m' + '%d'%N_bits +  '-bit to retrieval' + '\033[0m')
    
    #Define the data augmentation following the setup of SimCLR
    Augmentation = nn.Sequential(
        Kg.RandomResizedCrop(size=(sz, sz)),
        Kg.RandomHorizontalFlip(p=0.5),
        Kg.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1, p=0.8),
        Kg.RandomGrayscale(p=0.2),
        Kg.RandomGaussianBlur((int(0.1 * sz), int(0.1 * sz)), (0.1, 2.0), p=0.5))
    
    transform = transforms.ToTensor()
    
    class_name, train_img_list = get_data_list2('/content/drive/MyDrive/dataSets/CRC/train/')
    train_data_set=SingleData(class_name, train_img_list, transform)
    
    # trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=args.if_download, transform=transform)
    trainloader = T.utils.data.DataLoader(train_data_set, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=args.num_workers)
    # trainloader = T.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=args.num_workers)
    # testset = torchvision.datasets.CIFAR10(root=args.data_dir, train=False, download=args.if_download, transform=transforms.ToTensor())
    class Quantization_Head(nn.Module):
        def __init__(self, N_words, N_books, L_word, tau_q):
            super(Quantization_Head, self).__init__()
            self.fc = nn.Linear(512, N_books * L_word)
            nn.init.xavier_uniform_(self.fc.weight)
    
            # Codebooks
            self.C = T.nn.Parameter(Variable((T.randn(N_words, N_books * L_word)).type(T.float32), requires_grad=True))
            nn.init.xavier_uniform_(self.C)
    
            self.N_books = N_books
            self.L_word = L_word
            self.tau_q = tau_q
    
        def forward(self, input):
            X = self.fc(input)
            Z = Soft_Quantization(X, self.C, self.N_books, self.tau_q)
            return X, Z
    
    
    Q = Quantization_Head(N_words, N_books, L_word, tau_q)
    net = nn.Sequential(ResNet_Baseline(BasicBlock, [2, 2, 2, 2]), Q)
    net.load_state_dict(T.load('/content/drive/MyDrive/models/SPQ/199_32_0.1983_checkpoint.pth'))
    net.cuda(device)
    
    criterion = CQCLoss(device, batch_size, tau_cqc)
    
    optimizer = T.optim.Adam(net.parameters(), lr=3e-4, weight_decay=10e-6)
    scheduler = CosineAnnealingLR(optimizer, T_max=len(trainloader), eta_min=0, last_epoch=-1)
    
    MAX_mAP = 0.0
    mAP = 0.0
    
    
    mAP,acc = DoRetrieval2(device, args, net, Q.C)

32-bit to retrieval
*&$%#$%@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
['s1', 's2', 's3']


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Do Retrieval!!@!
*&$%#$%@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
['s1', 's2', 's3']
*&$%#$%@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
class test ['s1']
img test ['/content/drive/MyDrive/dataSets/CRC/temp heatmap tesnor/s1/s1_167_28_1927.png']
['s1']


Compute Query  100%|██████████| 1/1 [00:00<00:00,  1.34it/s]

%%%%2 tensor([[1., 0., 0.]]) ('/content/drive/MyDrive/dataSets/CRC/temp heatmap tesnor/s1/s1_167_28_1927.png',)


TypeError: ignored